In [1]:
import os
import json
import pandas as pd
import re

# EDA

In [2]:
dir = '../data/raw/20231212/funds/'
symbol_list = []
category = []
asset_class = []
inception_date = []
min_inv = []
exp_ratio = []
return_ytd = []
return_1_year = []
return_3_year = []
return_5_year = []
return_10_year = []
return_inception = []
return_bench_ytd = []
return_bench_1_year = []
return_bench_3_year = []
return_bench_5_year = []
return_bench_10_year = []
return_bench_inception = []
risk_level = []


for filename in os.listdir(dir):
    file_path = os.path.join(dir,filename)
    with open(file_path,'r') as f:
        fund_dict = json.load(f)
    pattern = r'\/([A-Z]+)\.json'
    match = re.search(pattern, file_path)
    symbol = match.group(1)
    symbol_list.append(symbol)
    
    category.append(fund_dict['key_fact_table']['Category'])
    asset_class.append(fund_dict['key_fact_table']['Asset class'])
    if 'Inception date' in fund_dict['key_fact_table']:
        inception_date.append(fund_dict['key_fact_table']['Inception date'])
    else:
        inception_date.append(None)

    if 'risk_level' in fund_dict:
        risk_level.append(fund_dict['risk_level'])
    else:
        risk_level.append(None)

    if 'perf_table' in fund_dict:
        perf_table = fund_dict['perf_table']

        if symbol in perf_table:
            symbol_key = symbol
        elif symbol+'1' in perf_table:
            symbol_key = symbol + '1'
        elif symbol+'2' in perf_table:
            symbol_key = symbol+'2'
        elif symbol+'4' in perf_table:
            symbol_key = symbol+'4'
        else:
            symbol_key = symbol+' (Market price)'
        
        if 'Benchmark1' in perf_table:
            benchmark_key = 'Benchmark1'
        elif 'Benchmark3' in perf_table:
            benchmark_key = 'Benchmark3'
        else:
            benchmark_key = 'Benchmark'

        for row in zip(perf_table['index'],perf_table[symbol_key],perf_table[benchmark_key]):
            if row[0]=="YTD":
                return_ytd.append(row[1])
                return_bench_ytd.append(row[2])
            if row[0]=="1-yr":
                return_1_year.append(row[1])
                return_bench_1_year.append(row[2])
            if row[0]=="3-yr":
                return_3_year.append(row[1])
                return_bench_3_year.append(row[2])
            if row[0]=="5-yr":
                return_5_year.append(row[1])
                return_bench_5_year.append(row[2])
            if row[0]=="10-yr":
                return_10_year.append(row[1])
                return_bench_10_year.append(row[2])
            if row[0]=="Since inception":
                return_inception.append(row[1]) 
                return_bench_inception.append(row[2])
    else:
        return_ytd.append(None)
        return_bench_ytd.append(None)
        return_1_year.append(None)
        return_bench_1_year.append(None)
        return_3_year.append(None)
        return_bench_3_year.append(None)
        return_5_year.append(None)
        return_bench_5_year.append(None)
        return_10_year.append(None)
        return_bench_10_year.append(None)
        return_inception.append(None)
        return_bench_inception.append(None)

    if 'min_investment' in fund_dict:
        min_inv.append(fund_dict['min_investment'])
    else:
        min_inv.append(None)
    
    if 'exp_ratio' in fund_dict:
        exp_ratio.append(fund_dict['exp_ratio'])
    else:
        exp_ratio.append(None)    

In [3]:
table_col = ['symbol_list','category','asset_class','inception_date','min_inv','exp_ratio','return_ytd','return_1_year','return_3_year','return_5_year','return_10_year','return_inception','return_bench_ytd','return_bench_1_year','return_bench_3_year','return_bench_5_year','return_bench_10_year','return_bench_inception','risk_level']

fund_df = pd.DataFrame(columns=table_col,data=list(zip(symbol_list,category,asset_class,inception_date,min_inv,exp_ratio,return_ytd,return_1_year,return_3_year,return_5_year,return_10_year,return_inception,return_bench_ytd,return_bench_1_year,return_bench_3_year,return_bench_5_year,return_bench_10_year,return_bench_inception,risk_level)))

In [6]:
fund_df[fund_df['min_inv'].isna()]

,symbol_list,category,asset_class,inception_date,min_inv,exp_ratio,return_ytd,return_1_year,return_3_year,return_5_year,return_10_year,return_inception,return_bench_ytd,return_bench_1_year,return_bench_3_year,return_bench_5_year,return_bench_10_year,return_bench_inception,risk_level
106,VPMCX,Large Growth,Domestic Stock - More Aggressive,11/01/1984,None,0.38%,22.27%,15.31%,9.97%,11.29%,12.66%,13.42%,20.80%,13.84%,9.76%,12.51%,11.82%,11.38%,5\n
121,VHCOX,Mid Growth,Domestic Stock - More Aggressive,08/14/1995,None,0.43%,18.67%,12.49%,7.42%,10.67%,12.20%,12.18%,17.00%,9.98%,0.43%,10.05%,10.11%,9.42%,5\n
129,VPMAX,Large Growth,Domestic Stock - More Aggressive,11/12/2001,None,0.31%,22.35%,15.39%,10.04%,11.37%,12.74%,10.83%,20.80%,13.84%,9.76%,12.51%,11.82%,8.69%,5\n
134,VFSAX,—,—,—,None,None,None,None,None,None,None,None,None,None,None,None,None,None,5\n
154,VPCCX,Large Blend,Domestic Stock - General,12/09/2004,None,0.46%,16.70%,10.23%,9.94%,10.46%,11.66%,10.65%,21.07%,13.95%,8.79%,12.40%,11.72%,9.67%,4\n
246,VHCAX,Mid Growth,Domestic Stock - More Aggressive,11/12/2001,None,0.36%,18.75%,12.57%,7.49%,10.75%,12.28%,11.05%,17.00%,9.98%,0.43%,10.05%,10.11%,9.38%,5\n


In [4]:
fund_df[fund_df['risk_level'].isna()]

,symbol_list,category,asset_class,inception_date,min_inv,exp_ratio,return_ytd,return_1_year,return_3_year,return_5_year,return_10_year,return_inception,return_bench_ytd,return_bench_1_year,return_bench_3_year,return_bench_5_year,return_bench_10_year,return_bench_inception,risk_level


# Cleaning Helper Functions

In [ ]:
def perc_to_decimal(perc_string):
    return round(float((perc_string.strip().replace('%',''))/100),2)